In [33]:
from openai import OpenAI
import weave
from weave import Model
from pydantic import field_validator
from configparser import ConfigParser
import json

In [6]:
config_parser = ConfigParser()
config_parser.read("config.cfg")
LLAMA_KEY = config_parser.get("DEFAULT", "LLAMA_KEY")

In [44]:
class Llama(Model):

    model_name : str
    api_key : str
    prompt : str

    @property
    def api(self):
        return OpenAI(
            api_key=self.api_key, 
            base_url="https://api.llama-api.com"
        )
    

    def format(self, payload : dict, params : dict, **kwargs) -> dict:

        prompt = self.prompt.format(**payload)
        return {
            "messages": [
                {"role": "user", "content": prompt},
            ],
            **params,
            **kwargs
        }

    @weave.op()
    def predict(self, prompt: str, params : dict, **kwargs):
        response = self.api.chat.completions.create(
            model=self.model_name, 
            **self.format(prompt, params, **kwargs)
        )
        result = response.choices[0].message.content

        if result is None:
            raise ValueError("No response from model")
        return result

In [60]:
PROMPT_TEMPLATE = """You will be given an example consisting of a context and a question to answer. The answer should always be of this form "The N V the N", where N stands for a single word that is a noun, and V stands for a single word that is a verb. 
Here are two samples:

        "Context": "The student the man saw is happy",
        "Question": "Who saw who?",
        "Answer": "The man saw the student.",


        "Context": "The teacher the student saw hit is happy",
        "Question": "Who saw who?",
        "Answer": "The student saw the teacher.",


Context: {context}
Question: {question}

Now answer the question:
"""

In [61]:
example = {
    "context": "The dog the cat saw was happy",
    "question": "Who saw who?",
    "A": "The cat saw the dog.",
    "level": "1"
}

In [62]:
model = Llama(
    name="llama-7b-chat",
    description="Weave model for Llama",
    model_name="llama-7b-chat",
    api_key=LLAMA_KEY,
    prompt=PROMPT_TEMPLATE
)

In [58]:
weave.init("first_try")

Logged in as Weights & Biases user: nthomsen.
View Weave data at https://wandb.ai/cbs-nlp/first_try/weave


In [63]:
output = model.predict(
    example,
    params={"max_tokens": 100}
)

🍩 https://wandb.ai/cbs-nlp/first_try/r/call/9f0b274b-e97f-40d5-9cfa-684b2816a422


In [65]:
output

'The cat saw the dog.'

In [64]:
output == example["A"]

False